# How to create a Bitcoin address

- Uses ECDSA keypair 
- Uses SHA-256 and RIPEMD-160 hashing algorithms
- Uses Base58Check encoding on the 25-byte binary bitcoin address


References: 
- https://en.bitcoin.it/wiki/Technical_background_of_version_1_Bitcoin_addresses
- https://stackoverflow.com/questions/443967/how-to-create-python-bytes-object-from-long-hex-string

### 1. Create a encryption key pair

In [1]:
from ecdsa import SigningKey, SECP256k1

sk = SigningKey.generate(curve=SECP256k1)
vk = sk.get_verifying_key()

In [2]:
vk.to_string()

b'?\x08\xc0o\xc4\x14\xf2\xa6Yi\xff\xabZ<\xe6\xa6\xe8s \xc8\xaf\xc2\xc9\xd8\xe4>\x8e\x1d*W\x14s\xcd\xfe]\x88\xf3\xacU\xac\xd5\xa5J\xed\xcf\x9d2}\xecJG\xef\xad\xc8\x06\x02Q\x92\xac\x1e,<\xbb\x00'

In [3]:
sk_hex = sk.to_string().hex() # Private Key
sk_hex

'68b73384c8a43cbdddde3a99795b302fd5adb17877fb4a63b16b18cd34d224ad'

In [4]:
vk_hex = vk.to_string().hex() # Public Key
vk_hex

'3f08c06fc414f2a65969ffab5a3ce6a6e87320c8afc2c9d8e43e8e1d2a571473cdfe5d88f3ac55acd5a54aedcf9d327dec4a47efadc806025192ac1e2c3cbb00'

### 2. Performing Hashing algorithms

In [5]:
# Stage 2: Perform first hash using SHA256
from Crypto.Hash import SHA256

hash_obj = SHA256.new(str.encode(vk_hex))
hash_1 = hash_obj.digest()

hash_1.hex() # len of 64

'7b4de75581a4f2aecdae0a06731efaca00cc1460db62b4b0fd243be250574c28'

In [6]:
# Stage 3: Perform 2nd hash using SHA RIPEMD-160 

from Crypto.Hash.RIPEMD import RIPEMD160Hash

RIPEMD160_hash_obj = RIPEMD160Hash()
RIPEMD160_hash_obj.update(hash_1)
hash_2 = RIPEMD160_hash_obj.digest()

hash_2.hex()

'734c4150fba7867974e70da7d3dc9f85d664b1b4'

In [7]:
# Stage 4: Add version byte in front of RIPEMD-160 hash (0x00 for Main Network)

hash_3 = "00" + hash_2.hex()

hash_3

'00734c4150fba7867974e70da7d3dc9f85d664b1b4'

In [8]:
# Stage 5: Perform SHA-256 Hash on extended RIPEMD-160 result

hash_4 = SHA256.new(str.encode(hash_3)).digest()

hash_4.hex()

'cfce04423dc2207ab2679477a0d1f227c77a9c92760ed06c8df92592e7249f62'

In [9]:
# Stage 6: Perform SHA-256 Hash on the previous result 

hash_5 = SHA256.new(hash_4).digest()

hash_5.hex()

'42d9b80f42c4fdf38b1a3e84175d107ede58f73b73176f73aeade531862e1240'

In [10]:
# Stage 7 : Take first 4 bytes to create the address checksum

checksum = hash_5[:4].hex()

checksum

'42d9b80f'

In [11]:
# Stage 8: Add the checksum to the extended RIPEMD-160 at the end of stage 4

bit_address = hash_3 + checksum 

print("25-byte binary bitcoin address:\n'{}'".format(bit_address))


25-byte binary bitcoin address:
'00734c4150fba7867974e70da7d3dc9f85d664b1b442d9b80f'


In [13]:
# Stage 9: Convert the result from a byte string into a base58 string
# using Base58Check encoding. This is the most commonly used bitcoin address format
import base58

address = base58.b58encode(bytes.fromhex(bit_address))

address

'1BWe4MvDUeq7yjsP5PcR64c85xAPm2spEa'

### 3. Is bitcoin address of a valid format? 

In [14]:
import base58
from Crypto.Hash import SHA256
from Crypto.Hash.RIPEMD import RIPEMD160Hash

def generate_checksum(hash_): 
    hash_1 = SHA256.new(str.encode(hash_)).digest().hex() 
    hash_2 = SHA256.new(str.encode(hash_1)).digest() 
    return hash_2[:4].hex() # Checksum

def is_valid_addr(pub_addr):
    binary_addr = base58.b58decode(pub_addr)
    checksum = binary_addr[-4:].hex()    
    if checksum == generate_checksum(binary_addr[:-4].hex()):
        return True
    else:
        return False

pub_addr = '185tFGDXv9tAVYiXWGDF6Ce4An5D95oWNP'  
is_valid_addr(pub_addr)

True